In [1]:
from google.colab import files
files.upload()

Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"gauchetaru","key":"4d22064d638b068c7e729d3f634eee12"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
#!/bin/bash
!kaggle datasets download muhammadbilalhaneef/sherlock-holmes-next-word-prediction-corpus

Dataset URL: https://www.kaggle.com/datasets/muhammadbilalhaneef/sherlock-holmes-next-word-prediction-corpus
License(s): CC0-1.0
sherlock-holmes-next-word-prediction-corpus.zip: Skipping, found more recently modified local copy (use --force to force download)


In [1]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [5]:
!unzip /content/sherlock-holmes-next-word-prediction-corpus.zip -d /content/


Archive:  /content/sherlock-holmes-next-word-prediction-corpus.zip
replace /content/Sherlock Holmes.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
file = open('/content/Sherlock Holmes.txt', 'r', encoding='utf-8')

In [3]:
lines = []
for i in file:
    lines.append(i)

In [4]:
data = ""
for i in lines:
    data = ' '.join(lines)

In [5]:
data = data.replace("\\'", "'").replace('\ufeff', '')

In [6]:
data = data.split()
data = ' '.join(data)

In [7]:
data[:1000]

"THE ADVENTURES OF SHERLOCK HOLMES Arthur Conan Doyle Table of contents A Scandal in Bohemia The Red-Headed League A Case of Identity The Boscombe Valley Mystery The Five Orange Pips The Man with the Twisted Lip The Adventure of the Blue Carbuncle The Adventure of the Speckled Band The Adventure of the Engineer's Thumb The Adventure of the Noble Bachelor The Adventure of the Beryl Coronet The Adventure of the Copper Beeches A SCANDAL IN BOHEMIA Table of contents Chapter 1 Chapter 2 Chapter 3 CHAPTER I To Sherlock Holmes she is always the woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false positio

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

In [9]:
pickle.dump(tokenizer, open('token.pkl', 'wb'))

In [10]:
sequence_data = tokenizer.texts_to_sequences([data])[0]

In [11]:
sequence_data[:10]

[1, 1561, 5, 129, 34, 647, 4498, 4499, 226, 5]

In [12]:
len(sequence_data)

105879

In [13]:
vocab_size = len(tokenizer.word_index) + 1

len(tokenizer.word_index) gives us number of unique words.

why +1? because we need space for the padding token.

each word has a number what about empty space.

we add +1 so the model can handle the "nothing here" token(Index 0) just like it handle real words.

as index 0 is reserved for padding:)

In [14]:
sequence = []

for i in range(10, len(sequence_data)):
    words = sequence_data[i - 10 : i + 1] #this creates a 4 token chunk, previous 3 tokens + current one
    sequence.append(words)

In [15]:
sequence = np.array(sequence)

In [16]:
sequence

array([[   1, 1561,    5, ...,  226,    5, 1562],
       [1561,    5,  129, ...,    5, 1562,    6],
       [   5,  129,   34, ..., 1562,    6,  827],
       ...,
       [8197, 1143,   84, ...,    1, 8198, 8199],
       [1143,   84,    9, ..., 8198, 8199, 3187],
       [  84,    9,   30, ..., 8199, 3187, 3186]])

In [17]:
x = []
y = []

for i in sequence:
    x.append(i[0:3])
    y.append(i[3])

x = np.array(x)
y = np.array(y)

In [18]:
x.shape, y.shape

((105869, 3), (105869,))

In [19]:
y = to_categorical(y, num_classes=vocab_size)

In [20]:
y.shape

(105869, 8200)

In [21]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization

weight_decay = 1e-5

Building the LSTM model

In [22]:
model = Sequential()

model.add(Embedding(vocab_size, 50, input_length = 10))
model.add(LSTM(128, dropout=0.2, return_sequences=True, kernel_regularizer=l2(weight_decay)))
BatchNormalization()
model.add(LSTM(128, dropout=0.2, kernel_regularizer=l2(weight_decay)))
BatchNormalization()
model.add(Dense(128, activation='relu', kernel_regularizer=l2(weight_decay)))
BatchNormalization()
Dropout(0.2)
model.add(Dense(vocab_size, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


first we have embedding.


we have already one hot encoded each word. they are sparse(alot of zeros) and equal distance like (dog-cat same as dog-airplane), no meaning.


model doesn't know that dog and cat are the same.


or 'walking' and 'run' are closer to each other than 'building'.



what embeddings do?

embeddings take each word and map into a smaller continuous space(like 50D or 100D), so in this space similar words live closer, relationships get preserved.

this way you can measure distances, clusters and relationships.

we talked about 50D or 100D vector, the vectors are trainable, they change so that the model's world aligns with the text of the training.



so, maybe the main spice is embeddings, where they teach the model the language of meaning, instead of raw IDs.

we just take 3 tokens into consideration.



now, the real memory unit

LSTM, it reads sequences step by step. it is better than vanilla RNN because it remembers long-term dependencies(remember vanishing gradient issue).


here, return_sequences = True, so that it return the hidden sequences for each word in the 3-word sequence, not just the last one.


take it like, we are watching a movie, and we want to keep all the timestamps, because we live by all the scenes not just the ending scene.

so,that by next time i watch it i can remember my favourite part.


that way, by returning sequences from the first LSTM, we can remember the whole flow by the next 'memory unit'(LSTM).



now, we have another LSTM stacked on top, this time, return_sequences=False(the default one) as it only outputs the final state.

take it like, i have read 3 words, here is distilled memory of it.

first LSTM captures the patterns across timestamps and the second one, compresses that into a single thought vector about the sequence.

dropout randomly 'forgets' 20% of the neurons during training.


why? to stop the model from memorizing your dataset(overfitting)


like forcing someone to recalling without much content so that they understand the pattern.

now, in the next layer we have relu activation function, a fully connected layer with 128 neurons. relu makes it non linear but for more complex mappings.  

like the brainstorming layers that reshapes the compressed thought from the LSTM into something useful for classification.

then in the final layer we have one neuron per word.

softmax turns logits into probabilities that sum to 1

highest probability -> boom next word

In [23]:
model.build(input_shape=(None, 10))   # None = batch size, 3 = sequence length

In [24]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 10, 50)         │       410,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 10, 128)        │        91,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8200)           │     1,057,800 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,707,544 (6.51 MB)

 Trainable params: 1,707,544 (6.51 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
# from tensorflow.keras.callbacks import ModelCheckpoint
# checkpoint = ModelCheckpoint("next_words.h5", monitor="loss", verbose=1, save_best_only=True)


In [26]:
from tensorflow.keras.optimizers import RMSprop


In [27]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [28]:
# from tensorflow.keras.callbacks import EarlyStopping

# early_stop = EarlyStopping(
#     monitor='val_loss',
#     patience = 3,
#     restore_best_weights=True
# )

In [29]:
model.fit(x, y, epochs=100, validation_split=0.2)

Epoch 1/100
2647/2647 ━━━━━━━━━━━━━━━━━━━━ 35s 11ms/step - accuracy: 0.0530 - loss: 6.8230 - val_accuracy: 0.0585 - val_loss: 6.3535
Epoch 2/100
2647/2647 ━━━━━━━━━━━━━━━━━━━━ 27s 10ms/step - accuracy: 0.0718 - loss: 6.1070 - val_accuracy: 0.0892 - val_loss: 6.1380
Epoch 3/100
2647/2647 ━━━━━━━━━━━━━━━━━━━━ 28s 10ms/step - accuracy: 0.0971 - loss: 5.7099 - val_accuracy: 0.1049 - val_loss: 6.0828
Epoch 4/100
2647/2647 ━━━━━━━━━━━━━━━━━━━━ 28s 10ms/step - accuracy: 0.1136 - loss: 5.4580 - val_accuracy: 0.1092 - val_loss: 6.1036
Epoch 5/100
2647/2647 ━━━━━━━━━━━━━━━━━━━━ 27s 10ms/step - accuracy: 0.1240 - loss: 5.2749 - val_accuracy: 0.1167 - val_loss: 6.1721
Epoch 6/100
2647/2647 ━━━━━━━━━━━━━━━━━━━━ 27s 10ms/step - accuracy: 0.1327 - loss: 5.1352 - val_accuracy: 0.1201 - val_loss: 6.2774
Epoch 7/100
2647/2647 ━━━━━━━━━━━━━━━━━━━━ 41s 10ms/step - accuracy: 0.1397 - loss: 4.9984 - val_accuracy: 0.1196 - val_loss: 6.3854
Epoch 8/100
2647/2647 ━━━━━━━━━━━━━━━━━━━━ 27s 10ms/step - accuracy: 

In [31]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [32]:
def predict_next_word_topk(model, tokenizer, text, max_sequence_len, k=5):
    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre')
    predicted_probs = model.predict(token_list, verbose=0)[0]

    # Pick top-k words
    top_k_idx = np.argsort(predicted_probs)[-k:]
    top_k_probs = predicted_probs[top_k_idx]
    top_k_probs = top_k_probs / np.sum(top_k_probs)  # normalize
    chosen_idx = np.random.choice(top_k_idx, p=top_k_probs)

    return tokenizer.index_word[chosen_idx]


In [33]:
seed_text = "the sun is shining in"
for i in range(20):
    next_word = predict_next_word_topk(model, tokenizer, seed_text, max_sequence_len=3, k=5)
    seed_text += " " + next_word
    print(seed_text)


the sun is shining in the
the sun is shining in the air
the sun is shining in the air of
the sun is shining in the air of the
the sun is shining in the air of the sort
the sun is shining in the air of the sort in
the sun is shining in the air of the sort in victoria
the sun is shining in the air of the sort in victoria street
the sun is shining in the air of the sort in victoria street with
the sun is shining in the air of the sort in victoria street with brown
the sun is shining in the air of the sort in victoria street with brown roads
the sun is shining in the air of the sort in victoria street with brown roads gaped
the sun is shining in the air of the sort in victoria street with brown roads gaped in
the sun is shining in the air of the sort in victoria street with brown roads gaped in the
the sun is shining in the air of the sort in victoria street with brown roads gaped in the room
the sun is shining in the air of the sort in victoria street with brown roads gaped in the room an

In [35]:
model.save("next_word_model.keras")   # Native Keras format


In [36]:
model.export("next_word_model")


Saved artifact at 'next_word_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 10), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 8200), dtype=tf.float32, name=None)
Captures:
  132726607625488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132726607623568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132726607625296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132726607624336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132726597126672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132726607624720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132726607625872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132728915645392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132726597126864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132726597124752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132726597124560: Tens

In [39]:
model.save("next_word_model.h5")
